## TMA4212 Project 2
#### Group: Nikolai Rasmus Sætren, Thomas Olaussen, Tiago Alexandre Alcobia Pereira


In [ ]:
%matplotlib inline
import numpy as np
import scipy as sp
from numpy import sin,cos
import pandas as pd #To format the output of the notebook.  
from IPython.display import display
import time
import matplotlib.pyplot as plt
newparams = {'figure.figsize': (8.0, 6.0), 'axes.grid': True,
             'lines.markersize': 8, 'lines.linewidth': 2,
             'font.size': 14}
from matplotlib import cm
plt.rcParams.update(newparams)

import warnings




In [ ]:
def Ak(h,a,b,c):

    A11 = a/h-b/2+h/3*c
    A12 = -a/h+b/2+h/6*c
    A21 = -a/h-b/2+h/6*c        # endra forteikn på alle b for ein test
    A22 = a/h+b/2+h/3*c

    return np.array([[A11,A12],[A21,A22]])

def Fk(f,h,elem):

    # Trapezoidal rule
    F1 = f(elem[0])/2*h
    F2 = f(elem[1])/2*h

    return np.array([F1,F2])

In [ ]:
def loc2glob(k,alpha):
    return k+alpha          # Numbering k from 0

In [ ]:

def conv_table(E,H):
    Rate=np.zeros(E.shape)
    Rate[:,1:]=np.log10(E[:,1:]/E[:,:-1])/np.log10(H[1:]/H[:-1])
    pd.options.display.float_format = '{:.8f}'.format
    df = pd.DataFrame(data={'h': H, 'Error L2': E[0] ,'Rate L2':Rate[0], 'Error H1' : E[1], 'Rate H1' : Rate[1]}) 
    display(df)

def plotError(E,H,order, title):
    # plt.figure()
    # plt.loglog(H,E,label=f'$p$ = {order:.3f}')
    # plt.loglog(H,E,'o')
    # plt.title(f'Convergence Errors')
    # plt.xlabel('$h$')
    # plt.ylabel('Error')
    # plt.legend()
    # plt.show()

    fig, ax = plt.subplots(2)
    fig.suptitle(title)

    ax[0].loglog(H,E[0],label=f'$p$ = {order[0]:.3f}')
    ax[0].loglog(H,E[0],'o')
    ax[0].legend(loc = "upper left")
    ax[0].set_title(r"$L^2$-Norm")

    ax[1].loglog(H,E[1],label=f'$p$ = {order[1]:.3f}')
    ax[1].loglog(H,E[1],'o')
    ax[1].legend(loc = "upper left")
    ax[1].set_title(r"$H^1$-Norm")

    # fig.text(0.5, 0.02, '$h$', ha='center')
    # fig.text(0.02, 0.5, 'Error', va='center', rotation='vertical')

    fig.add_subplot(111, frameon=False)
    plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
    plt.xlabel('$h$')
    plt.ylabel('Error')
    plt.grid(False)

    plt.tight_layout()
    plt.show()



In [ ]:
def normL2(u):
    u2 = lambda x: u(x)**2
    return np.sqrt(sp.integrate.quad(u2,0,1)[0])

def normH1(u,du):
    u2 = lambda x: u(x)**2
    du2 = lambda x: du(x)**2

    return np.sqrt(sp.integrate.quad(u2,0,1)[0]+sp.integrate.quad(du2,0,1)[0])

In [ ]:
# Just for ease Quality of Life

def bmatrix(a):
    """Returns a LaTeX bmatrix

    :a: numpy array
    :returns: LaTeX bmatrix as a string
    """
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    return '\n'.join(rv)

# A = np.array([[12, 5, 2], [20, 4, 8], [ 2, 4, 3], [ 7, 1, 10]])
# print(bmatrix(A) + '\n')

# B = np.array([[1.2], [3.7], [0.2]])
# print(bmatrix(B) + '\n')

# C = np.array([1.2, 9.3, 0.6, -2.1])
# print(bmatrix(C) + '\n')

In [ ]:
class BVP(object):
    def __init__(self, f=None, u = None,F = None, a = 1, b = 1,c = 1):
        self.f = f
        self.u = u
        self.a = a
        self.b = b
        self.c = c
        self.U = None
        self.grid = None
        self.F = F
    
    def solveFDM(self,grid, printA = False):
        
        f = self.f
        a = self.a
        b = self.b
        c = self.c

        Xk = grid
        Mi = len(Xk)                     # Number of nodes
        T = np.ndarray.transpose(np.array([Xk[:-1],Xk[1:]])) # Set of elements/triangulation
        H = Xk[1:]-Xk[:-1]               # Element sizes
        Mk = Mi-1                        # Number of elements

        A = np.zeros((Mi,Mi))
        F = np.zeros((Mi))

        if (self.f == None):

            if self.F == None:

                def Fk1(u,h,elem):
                    # Trapezoidal rule
                    F1 = -a*(u(elem[1])-u(elem[0]))/h + b*(u(elem[0])+u(elem[1]))/2 + c*u(elem[0])/2*h
                    F2 = +a*(u(elem[1])-u(elem[0]))/h - b*(u(elem[0])+u(elem[1]))/2 + c*u(elem[1])/2*h

                    return np.array([F1,F2])

                for k in range(0,Mk):
                    for alpha in range(2):
                        i = loc2glob(k,alpha)
                        for beta in range(2):
                            j = loc2glob(k,beta)
                            A[i,j] += Ak(H[k],a,b,c)[alpha,beta]
                        F[i] += Fk1(self.u,H[k],T[k])[alpha]

            else:

                Fk1 = self.F

                for k in range(0,Mk):
                    for alpha in range(2):
                        i = loc2glob(k,alpha)
                        for beta in range(2):
                            j = loc2glob(k,beta)
                            A[i,j] += Ak(H[k],a,b,c)[alpha,beta]
                        F[i] += Fk1(H[k],T[k])[alpha]

        else:
            for k in range(0,Mk):
                for alpha in range(2):
                    i = loc2glob(k,alpha)
                    for beta in range(2):
                        j = loc2glob(k,beta)
                        A[i,j] += Ak(H[k],a,b,c)[alpha,beta]
                    F[i] += Fk(f,H[k],T[k])[alpha]

        # Impose boundary condition
        # Dirichlet boundary conditions
        A[0,:] = 0;
        A[0,0] = 1
        F[0] = 0
        A[-1,:] = 0
        A[-1,-1] = 1
        F[-1] = 0

        if printA:
            with np.printoptions(precision=3):
                B = bmatrix(A)
                print(A,"\n")
                print(B)

        # print(A)
        # print(F)

        # Solve
        U = np.linalg.solve(A,F)
        return U

    def solve(self,grid, printA=False):
        self.grid = grid

        U = self.solveFDM(grid,printA)

        self.U = U

    def plotSolution(self,title=''):
        plt.figure()
        plt.plot(self.grid,self.U)
        plt.xlabel('x')
        plt.title(title)
        plt.show()

    def plotComparison(self,title='', line=False):
        x = np.linspace(0,1,100)

        plt.figure()
        plt.plot(self.grid,self.U)
        plt.plot(self.grid,self.u(self.grid),'o', color='orange')
        if line:
            plt.plot(x,self.u(x), color='orange')
        plt.legend(['Numerical','Exact'])
        plt.xlabel('x')
        plt.title(title)
        plt.show()

    def errorL2(self,grid):
        
        U = self.solveFDM(grid)

        def error(x):
            return self.u(x) - np.interp(x,grid,U)
        warnings.filterwarnings("ignore")
        E = normL2(error)
        warnings.filterwarnings("default")
        return E

    def errorH1(self,grid, du):
        
        U = self.solveFDM(grid)

        U_interp = lambda x: np.interp(x,grid,U)
        dU = lambda x: sp.misc.derivative(U_interp,x,dx=1e-7)

        def error(x):
            return self.u(x) - U_interp(x)
        
        def derror(x):
            return du(x) - dU(x)
        
        warnings.filterwarnings("ignore")
        E = normH1(error,derror)
        warnings.filterwarnings("default")
        return E


    def conv_error(self,grids,du):

        E = np.zeros((2,len(grids)))        # Error
        H = np.zeros(len(grids))        # h values

        for i,grid in enumerate(grids):
            E[0,i] = self.errorL2(grid)
            E[1,i] = self.errorH1(grid,du)
            H[i]   = grid[1]-grid[0]

        order = np.zeros(2)
        try:
            order[0] = np.polyfit(np.log(H), np.log(E[0]),1)[0] # Order of convergence
            order[1] = np.polyfit(np.log(H), np.log(E[1]),1)[0] # Order of convergence

        except:
            pass
        
        return E, H, order

    def conv(self,grids,du,plotTitle = ''):
        E,H,order = self.conv_error(grids,du)

        conv_table(E,H)
        plotError(E,H,order, plotTitle)
        

In [ ]:
a,b,c = 1,1,1

def u(x):
    return x*(1-x)

def du(x):
    return 1-2*x

def f(x):
    return 2*a+b*(1-2*x)+c*u(x)

test1 = BVP(f=f,u=u)

xk = np.linspace(0,1,4)

test1.solve(xk,True)
test1.plotComparison(r'$u(x) = x(1-x)$',line=True)

xk = np.linspace(0,1,20)

test1.solve(xk)
test1.plotComparison(r'$u(x) = x(1-x)$')


grids = []
for i in range(4,160,24):
    grids.append(np.linspace(0,1,i))

test1.conv(grids,du, r'$u(x) = x(1-x)$')



In [ ]:
a,b,c = 1,1,1

def u(x):
    return sin(2*np.pi*x)

def du(x):
    return sp.misc.derivative(u,x,dx=1e-7)

def d2u(x):
    return sp.misc.derivative(du,x,dx=1e-7)

def f(x):
    return -a*d2u(x)+b*du(x)+c*u(x)

# Xk = np.array([0, 0.1, 0.3, 0.6, 0.8, 0.9, 0.95, 0.97, 1])  # Nodes, elements K(i) = (x(i-1), x(i))

Xk = np.linspace(0,1,20)


test1 = BVP(f,u)
test1.solve(Xk)
test1.plotComparison(r'$u(x) = \sin(2\pi x)$')

# test1.errorL2(Xk)

In [ ]:
a,b,c = 1,1,1

def w1(x):
    chk = (x <= np.sqrt(2)/2)

    return x*np.sqrt(2)*chk + (1-x)/(1-np.sqrt(2)/2)*(1-chk)

def dw1(x):
    chk = (x <= np.sqrt(2)/2)

    return np.sqrt(2)*chk - 1/(1-np.sqrt(2)/2)*(1-chk)

def w2(x):
    return x-x**(3/4)

def dw2(x):
    return 1-3/4*x**(-1/4)


Xk = np.linspace(0,1,20)


w1FDM = BVP(u = w1)
w2FDM = BVP(u = w2)

# w1FDM.solve(Xk)
# w1FDM.plotComparison()

# w2FDM.solve(Xk)
# w2FDM.plotComparison()


grids = []
for i in range(100,4000,100):
    grids.append(np.linspace(0,1,i))

# w1FDM.conv(grids,dw1, r'Errors: $w_1$')
w2FDM.conv(grids,dw2,r'Errors: $w_2$')




In [ ]:
def f1(x):
    return x**(-2/5)

def f2(x):
    return x**(-7/5)

def Fk1(h,elem):

    F = lambda x: x**(3/5)

    F1 = -1/h*2/3*(F(elem[1])-F(elem[0]))
    F2 = +1/h*2/3*(F(elem[1])-F(elem[0]))

    return np.array([F1,F2])

f1FDM = BVP(f = f1, b = -1)
f2FDM = BVP(F = Fk1, b= -1)
f2Alt = BVP(f  = f2, b = -1)

r = 0.95
M = 100

Xk = np.array([0] + [r**(M-i) for i in range(1,M+1)] )


f1FDM.solve(Xk)
f1FDM.plotSolution()

f2FDM.solve(Xk)
f2FDM.plotSolution()

# f2Alt.solve(Xk)
# f2Alt.plotSolution()



In [ ]:
def errorL2(grid,ref, f = None, F = None, b = 1):
    comp = BVP(f=f,F=F, b = b)
    comp.solve(grid)

    Ucomp = lambda x: np.interp(x,comp.grid,comp.U) 
    Uref = lambda  x: np.interp(x,ref.grid,ref.U) 

    diff = lambda x: Ucomp(x)-Uref(x)

    warnings.filterwarnings("ignore")
    E = normL2(diff)
    warnings.filterwarnings("default")
    
    return E

def errorH1(grid,ref,f = None, F = None, b = 1):
    comp = BVP(f=f,F=F, b = b)
    comp.solve(grid)

    Ucomp = lambda x: np.interp(x,comp.grid,comp.U) 
    Uref = lambda  x: np.interp(x,ref.grid,ref.U) 

    dUcomp = lambda x: sp.misc.derivative(Ucomp,x,dx=1e-7)
    dUref = lambda x: sp.misc.derivative(Uref,x,dx=1e-7)

    diff = lambda x: Ucomp(x)-Uref(x)
    ddiff = lambda x: dUcomp(x)-dUref(x)
    
    warnings.filterwarnings("ignore")
    E = normH1(diff,ddiff)
    warnings.filterwarnings("default")

    return E


In [ ]:
Xref = np.linspace(0,1,2000)

f1Ref = BVP(f = f1, b = -1)
f2Ref = BVP(F = Fk1, b = -1)

f1Ref.solve(Xref)
f2Ref.solve(Xref)


N = 50
Xlin = np.linspace(0,1,N)
h = 1/(N-1)


R = [0.98-0.03*i for i in range(7)]
R.reverse()
gridR = []

for i,r in enumerate(R):
    M = N-1
    gridR.append(np.array([0] + [r**(M-i) for i in range(1,M+1)] ))


EL2lin = np.zeros(2)
EH1lin = np.zeros(2)
EL2lin[0] = errorL2(Xlin,f1Ref,f=f1, b= -1)
EL2lin[1] = errorL2(Xlin,f2Ref,F=Fk1, b= -1)
EH1lin[0] = errorH1(Xlin,f1Ref,f=f1, b= -1)
EH1lin[1] = errorH1(Xlin,f2Ref,F=Fk1, b= -1)

EL2 = np.zeros((2,len(R)))
EH1 = np.zeros((2,len(R)))

for i,grid in enumerate(gridR):
    EL2[0,i] = errorL2(grid,f1Ref,f=f1, b= -1)
    EL2[1,i] = errorL2(grid,f2Ref,F=Fk1, b= -1)
    EH1[0,i] = errorH1(grid,f1Ref,f=f1, b= -1)
    EH1[1,i] = errorH1(grid,f2Ref,F=Fk1, b= -1)



In [ ]:

def errorTable(EL2,EH1,R):
    pd.options.display.float_format = '{:.8f}'.format
    df = pd.DataFrame(data={'r': R, 'Error L2: f1': EL2[0], 'Error L2: f2': EL2[1], 'Error H1: f1' : EH1[0], 'Error H1: f2' : EH1[1]}) 
    display(df)

In [ ]:

errorTable(EL2,EH1,R)


pd.options.display.float_format = '{:.8f}'.format
df = pd.DataFrame(data={'h': [h], 'Error L2: f1': [EL2lin[0]], 'Error L2: f2': [EL2lin[1]], 'Error H1: f1' : [EH1lin[0]], 'Error H1: f2' : [EH1lin[1]]}) 
display(df)

In [ ]:
a,b,c = 1,1,1

def u(x):
    return x*(1-x)

def du(x):
    return 1-2*x

def f(x):
    return 2*a+b*(1-2*x)+c*u(x)

test1 = BVP(f=f,u=u)

xk = np.linspace(0,1,4)

test1.solve(xk,True)
